In [9]:
!pip install opencv-python
import cv2
import numpy as np
import matplotlib.pyplot as plt

import-im6.q16: unable to open X server `172.17.0.39:0.0' @ error/import.c/ImportImageCommand/359.


In [2]:
# we're gonna use corner detection (SHi-Tomasi)

In [3]:
corner_track_params = dict(maxCorners = 10,qualityLevel=0.2,minDistance=5,blockSize=2)

In [4]:
#from the lukas Kanade method
#small windows might be too sensitive to noise and miss some of the motion(taking it for noise)
lk_params = dict(winSize=(200,200),maxLevel=2,criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,0.83))

In [10]:
cap = cv2.VideoCapture(0)
#'Camera Roll/vid_jason.mp4',0

ret,prev_frame = cap.read()

prev_gray = cv2.cvtColor(prev_frame,cv2.COLOR_BGR2GRAY)

#Points to track
prev_pts = cv2.goodFeaturesToTrack(prev_gray,mask=None,**corner_track_params)

mask = np.zeros_like(prev_frame)

while True:
    
    ret,frame = cap.read()
    
    frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    next_pts,status,err = cv2.calcOpticalFlowPyrLK(prev_gray,frame_gray,prev_pts,None,**lk_params)
    
    good_new = next_pts[status==1]
    good_prev = prev_pts[status==1]
    
    for i, (new,prev) in enumerate(zip(good_new,good_prev)):
        
        x_new,y_new = new.ravel()
        x_prev,y_prev = prev.ravel()
        
        mask = cv2.line(mask,(x_new,y_new),(x_prev,y_prev),(0,255,0),3)
        
        frame = cv2.circle(frame,(x_new,y_new),8,(0,0,255),-1)
        
    img = cv2.add(frame,mask)
    cv2.imshow('tracking',img)
    
    k = cv2.waitKey(1)
    if k == 27:
        break
        
        
    prev_gray = frame_gray.copy()
    prev_pts = good_new.reshape(-1,1,2)
    
    
cv2.destroyAllWindows()
cap.release()

TypeError: VideoCapture() takes at most 1 argument (2 given)

In [10]:
# The code above tracks sparse points

In [12]:
###
# DENSE OPTICAL FLOW

In [36]:
cap = cv2.VideoCapture('Camera Roll/vid_billy.mp4')

ret,frame1 = cap.read()

prev_img = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] = 255

while True:
    
    ret, frame2 = cap.read()
    
    next_img = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    
    flow = cv2.calcOpticalFlowFarneback(prev_img,next_img,None,0.5,3,15,3,5,1.2,0)
    
    # get polar coordinates
    mag,ang = cv2.cartToPolar(flow[:,:,0],flow[:,:,1],angleInDegrees=True)
    
    hsv_mask[:,:,0] = ang/2
    
    hsv_mask[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    
    bgr = cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)
    cv2.imshow('frame',bgr)
    
    k = cv2.waitKey(10) & 0xFF
    if k == 27:
        break
        
    prev_img = next_img
        
cap.release()
cv2.destroyAllWindows()